In [1]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import pytorch_cos_sim
model = SentenceTransformer("stsb-xlm-r-multilingual", device="cuda:0")

from glob import glob
from bert_utils import *
from config import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

****** SEED fixed : 42 ******




In [2]:
train = pd.read_csv(data_path+"train.csv")
test = pd.read_csv(data_path+"test.csv")
df = pd.concat([train, test]).reset_index(drop=True)
train_shape = train.shape[0]
df["clean_text"] = df["text"].map(lambda x: clean_text(x))

In [3]:
# 対象とするデータの読み込み --
corpus_paths = glob(f"{input_root}*.feather")
Debug_print(corpus_paths)

corpus_dfs = []
for corpus_path in corpus_paths:
    _df = pd.read_feather(corpus_path)
    _df = _df.reset_index(drop=False, names="id")
    _df["id"] = corpus_path.split("/")[-1].split(".")[0] + "_" + _df["id"].astype(str)
    corpus_dfs.append(_df)
corpus_df = pd.concat(corpus_dfs)

['./input/news4vip.feather', './input/newsplus.feather', './input/livejupiter.feather']


In [4]:
df_embeddings = model.encode(df["clean_text"].values.tolist(), convert_to_tensor=True)

In [5]:
df_embeddings

tensor([[ 0.3889, -0.2023,  0.1946,  ..., -0.0673, -0.8024,  0.0413],
        [ 0.1257,  0.2241,  0.4860,  ...,  0.0082,  0.2906,  0.0625],
        [-0.2709,  0.0892, -0.6683,  ...,  0.3698, -0.7946, -0.3370],
        ...,
        [ 0.2395,  0.3175,  0.5918,  ..., -0.1631, -0.0500,  0.2718],
        [-0.2001,  0.4499,  0.4081,  ...,  0.3871,  0.0518,  0.3478],
        [ 0.0224, -0.3445,  0.4316,  ...,  0.4233, -0.7020,  0.5813]],
       device='cuda:0')

In [6]:
corpus_embeddings = model.encode(corpus_df["clean_text"].values.tolist(), convert_to_numpy=True)

In [ ]:
np.save("./input/corpus_embeddings.npy", arr=corpus_embeddings)

In [ ]:
# for i in range(df_embeddings.shape[0]):
#     t = df_embeddings[i, :]
#     score = pytorch_cos_sim(t, corpus_embeddings)
#     break